In [1]:
!python -m pip install mysql-connector-python


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import mysql.connector
import pickle
import numpy as np
from xgboost import XGBClassifier

# Step 1: Connect to DB
conn_config = {
    'host': '127.0.0.1',
    'user': 'centralAuth',
    'password': 'messi10',
    'database': 'fl_database'
}
conn = mysql.connector.connect(**conn_config)
cursor = conn.cursor()

# Step 2: Fetch client updates
cursor.execute("""
    SELECT client_id, model_blob, accuracy, loss, macro_f1, recall_minority, f1_minority, f1_majority, fit_status
    FROM client_updates
    WHERE model_id = 2 AND round_num = 2
""")
client_updates = cursor.fetchall()

print("Fetched model updates from", len(client_updates), "clients.\n")

Fetched model updates from 4 clients.



In [7]:
param_list = []
performance_list = []

for row in client_updates:
    client_id, model_blob, accuracy, loss, macro_f1, recall_minority, f1_minority, f1_majority, fit_status = row
    
    model = pickle.loads(model_blob)
    params = model.get_params()
    param_list.append(params)
    performance_list.append({
        "client_id": client_id,
        "accuracy": accuracy,
        "macro_f1": macro_f1,
        "model": model,
        "params": params
    })
    
    print(f"Client ID: {client_id}")
    print("Params:", params)
    print(f"  Accuracy: {accuracy:.4f}, Macro F1: {macro_f1:.4f}, Fit Status: {fit_status}\n")

Client ID: 2
Params: {'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.88, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'logloss', 'feature_types': None, 'feature_weights': None, 'gamma': 2.02, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.2, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 7, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 300, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': 4, 'subsample': 0.91, 'tree_method': None, 'validate_parameters': None, 'verbosity': None, 'use_label_encoder': False}
  Accuracy: 0.6864, Macro F1: 0

In [8]:
# Step 4: Select best model based on macro F1 or accuracy
best_model_info = max(performance_list, key=lambda x: x["macro_f1"])  # or use "accuracy"
best_params = best_model_info["params"]
best_client_id = best_model_info["client_id"]

print("🎯 Best Model Based on Macro F1:")
print(f"  Client ID: {best_client_id}")
print(f"  Macro F1: {best_model_info['macro_f1']:.4f}")
print("  Selected Params:", best_params)


🎯 Best Model Based on Macro F1:
  Client ID: 1
  Macro F1: 0.6072
  Selected Params: {'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.88, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'logloss', 'feature_types': None, 'feature_weights': None, 'gamma': 2.02, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.2, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 7, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 300, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': 4, 'subsample': 0.91, 'tree_method': None, 'validate_parameters': None, 'verbosity': N

In [9]:
# Step 5: Create final global model using best params
global_model = XGBClassifier(**best_params)

# Optionally, fit it to central/global data if you have one:
# global_model.fit(X_global, y_global)

print("\n✅ Global model defined using best client’s parameters.")
global_model


✅ Global model defined using best client’s parameters.


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.88, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=2.02,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, ...)

In [10]:
model =XGBClassifier(
    objective='binary:logistic',
    eval_metric='aucpr',
    scale_pos_weight=5,              # very important for imbalance
    learning_rate=0.1,               # slower learning, less overfit
    max_depth=7,                     # same
    n_estimators=300,
    gamma=2,
    colsample_bytree=0.88,
    subsample=0.8,                   # add this
    min_child_weight=3,             # add this
    random_state=42
)


In [11]:
global_blob = pickle.dumps(model)

In [12]:
import mysql.connector
import pickle
from sklearn.linear_model import LogisticRegression





conn_config = {
    'host': '127.0.0.1',
    'user': 'centralAuth',
    'password': 'messi10',
    'database': 'fl_database'
}
conn = mysql.connector.connect(**conn_config)
cursor = conn.cursor()


query = "INSERT INTO central_updates (model_id, version, model_blob, model_name) VALUES (%s, %s, %s, %s)"
cursor.execute(query, (1, 2, global_blob, 'XGBoost'))

# cursor.execute(f"INSERT INTO central_models (model_id, version, model_blob) VALUES (1, 1, {model_blob})")
conn.commit()
cursor.close()
conn.close()


**example clients**

In [59]:
import mysql.connector
import pickle
from sklearn.linear_model import LogisticRegression





conn_config = {
    'host': '127.0.0.1',
    'user': 'centralAuth',
    'password': 'messi10',
    'database': 'fl_database'
}
conn = mysql.connector.connect(**conn_config)
cursor = conn.cursor()


cursor.execute("SELECT model_blob FROM central_models WHERE model_id = %s AND version = %s", (1, 1))
result = cursor.fetchone()
cursor.close()
conn.close()

if result is None:
    raise Exception("Model not found in database.")
model_blob = result[0]
model_blob


b'\x80\x04\x95K\x01\x00\x00\x00\x00\x00\x00\x8c\x1esklearn.linear_model._logistic\x94\x8c\x12LogisticRegression\x94\x93\x94)\x81\x94}\x94(\x8c\x07penalty\x94\x8c\x02l2\x94\x8c\x04dual\x94\x89\x8c\x03tol\x94G?\x1a6\xe2\xeb\x1cC-\x8c\x01C\x94G?\xf0\x00\x00\x00\x00\x00\x00\x8c\rfit_intercept\x94\x88\x8c\x11intercept_scaling\x94K\x01\x8c\x0cclass_weight\x94\x8c\x08balanced\x94\x8c\x0crandom_state\x94N\x8c\x06solver\x94\x8c\x05lbfgs\x94\x8c\x08max_iter\x94M\xf4\x01\x8c\x0bmulti_class\x94\x8c\x0bmultinomial\x94\x8c\x07verbose\x94K\x00\x8c\nwarm_start\x94\x89\x8c\x06n_jobs\x94N\x8c\x08l1_ratio\x94N\x8c\x10_sklearn_version\x94\x8c\x051.5.2\x94ub.'

In [60]:
if isinstance(model_blob, str):
    model_blob = model_blob.encode('latin1')  # Or 'utf-8' if needed
model = pickle.loads(model_blob)
model

LogisticRegression(class_weight='balanced', max_iter=500,
                   multi_class='multinomial')